# Synthesis - Day 1
In today's lecture, we are going to review some of the synthesis techniques, we covered in Intro to Audio Tech I.


In [ ]:
import numpy as np
from IPython.display import Audio
#%run hide_toggle.ipynb
from scipy.io.wavfile import read
from scipy import signal
import matplotlib.pyplot as plt


## Additive Synthesis

When we create new sounds out of the *addition* of basic sinusoids, that's called additive synthesis. Let's review some basic "recipes" for some familiar waveforms that are the result of basic additive synthesis. That is, we'll build everything by combining simple sinusoids.

Recall the basic formula for a real-valued sinusoid:

$$ A\ sin(\omega_{0}t + \phi)$$




To perform additive synthesis, we can just add our *equal size* numpy signal arrays

The resulting audio of two sounds added together will always equal (percpetually) those two independent sounds now sounding simultaneously. (However, if the sinusoids are integer multiples of each other, they may be perceived as a single sound with multiple harmonics.)

# Complex Tones

Remember: a complex tone is any signal that can be described as periodic, and as made up of more than one sinusoidal component. These individual frequencies or sinusoidal components are called partials.

## Harmonic signals

* If a complex tone has partials that are related by integer ratios, it is a harmonic signal. 
* The lowest harmonic is called the fundamental frequency and is usually the perceived pitch. 


![Alt text](https://upload.wikimedia.org/wikipedia/commons/b/b7/Harmonic_spectra_theoretical_x_y.png)

## Inharmonic signals

* Inharmonic signals generate complex tones where the partials are not related by integer ratio. 
* E.g.: bells, tympani. Imprecise overall pitch; can have pitch corresponding to dominant partial or even several pitches
*  These generate waveforms that are not clearly periodic, but can still be described as a series of superimposed sinusoids.

In [ ]:
Image(url='https://upload.wikimedia.org/wikipedia/commons/f/f7/Bell_inharmonic_spectrum.png', width=600)

### Noise & Non-Musical Sounds

* E.g., white noise, cymbals
* Dog barking
* Aperiodic signal (random temporal representation - waveform does not clearly repeat). **Does not convey the perception of pitch.**

* Although some sound sources have single-frequency components, most sound sources
produce a very disordered and random waveform of amplitude versus time. 

**ALL complex sounds can be represented as a sum of sinusoidal signals or *components*.** 
The process by which a complex wave is decomposed (broken up) into a set of component sinusoids is referred to as Fourier analysis.

In [ ]:
from IPython.display import Image
Image(url="https://hackster.imgix.net/uploads/attachments/537944/image_KipMQodplQ.png?auto=compress%2Cformat&w=1280&h=960&fit=max")

We can create simple "recipes" for the most basic of complex waveforms, which are the combination of specific combinations of a fundamental sinusoid and various integer multiples (in Hz)

Below is a table for comparing a pure tone, and 3 complex (artificial) waveforms, showing which harmonics are present and at which amplitudes:

| Waveshape | a1 | a2 | a3| a4| a5| a6| a7 | a8 | a9 | aN | General Rule |
| ---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |:---: | :------------------: |
| Sine | 1 | 0 | 0 | 0 | 0 |0 | 0 | 0 | 0 | ... | $f0$ only |
| Sawtooth | 1 | 1/2 | 1/3 | 1/4 | 1/5 | 1/6 | 1/7 | 1/8 | 1/9 | ... |$1/x$ |
| Square | 1 | 0 | 1/3 | 0 | 1/5 | 0 | 1/7 | 0 | 1/9 | ... |$1/x$ for odd $x$ |
| Triangle |  1 | 0 | -1/9 | 0 | 1/25 | 0 | -1/49 | 0 | 1/81 | ... |$1/x^2$ for odd $x$, alternating + and - | 

Let's build a sawtooth wave using our genSine() function

Let's build a saw another way using loops

Let's do it again using numpy functions and a zipped for loop

# Envelopes
In sound and music, an **envelope** describes how a sound changes over time. 

### ADSR

One of the most common kinds of envelopes for modeling "real" sounds has four components: Attack, Decay, Sustain, Release.

Remember:
The attack, decay, and release are all technically **time parameters** while sustain is a **level parameter**.  

A --> The attack is the amount of time from sound onset (e.g., key strike) to peak amplitude  
D --> The decay is the amount of time to descend from the peak amplitude to the sustain level  
S --> Level of sustain  
R --> Time from sound offset (e.g., key release) to zero amplitude.

Since ADSR envelopes describe how the basic outline of a sound's amplitude varies *over time*, ADSR is a type of *temporal envelope*


In [ ]:
from IPython.display import Image, Audio
Image('../AudioTechII-Claire/images/ADSR.jpg', width = 600, embed=True)

We can apply amplitude envelopes to audio in Python by multiplying our original audio array with an array containing the envelope. 

The length of your amplitude envelope must be the same size as your original audio file

## Modulation - Amplitude LFOs

Just like an envelope function, we can modify one signal by multiplying it with another which will affect its amplitude over time. So a naturally smooth, time-varying signal like a sinusoid is great for creating a tremolo-like effect. 
One way of achieving this is with an LFO (low frequency oscillator)

For amplitude modulation - we will multiply the carrier and the modulator signals

$$s(t) = (1 + m(t)) \times A_c\sin(2\pi f_c t)$$
...where $A_c$ is the carrier amplitude, 1 is the DC offset, and $m(t)$ is the modulator signal.

The modulator is always a periodic or quasi-periodic oscillator, with a clearly defined frequency, waveshape, and amplitude.
By taking a signal oscillating at a very low frequency, and using that as our modulator frequency, we can alter a carrier signal's amplitude in a regular way. This is known as an LFO.

* The frequency of the modulator affects the rate of change of the carrier's amplitude. I.e., increasing the frequency of the modulator will increase the tremolo effect in the modulated signal.

* The amplitude (or modulator index) of the modulator affects the *depth* of change of the carrier's amplitude

* The shape of the modulator affects the regular time-varying amplitude shapes in the carrier's amplitude

The modulator can be any periodic signal (or waveform), and the carrier can be any incoming signal.
Let's try with a square lfo on a loaded audio file

## AM Synthesis

If we increase the frequency of the modulator, we create a new complex sound with additional frequency components. 

Recall with modulators above 20 Hz, we will get sidebands at carrier_freq $\pm$ modulator_freq along with a "preserved" carrier.

## Ring modulation

Ring modulation is the same as amplitude modulation, but it specifically uses 2 bipolar signals. 

Notice this affects the magnitude of the sidebands and the presence of the carrier frequency in the final signal.